In [1]:

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn import linear_model
from sklearn.linear_model import RidgeClassifier

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


What cost (error) function are you using?

- According to google, RidgeClassifier from sklearn uses least linear squares

What is the optimal ridge penalty parameter (alpha) based on training data?

- alpha = 0.1
   
What is the optimal ridge penalty parameter on your validation data?
   
- alpha = 0.0

What is your final model error? (Be sure to report the most honest error
assessment).

- Assuming the error is just 100-score for the test set, the most honest model error is about 0.78. When I repeat the train test split steps like it says in the next question, this varies. But for this 'round' it was about .22, or 22% error.

- Also, every time I restart my kernel and rerun cells I get different score values so if the scores are different than what I wrote down in the little notes to myself in the code i promise i was telling the truth at the time that i wrote them!

If you redo your train/validation splits, do you get the same answers?

- I get the same optimal ridge penalties but the values for the actual scores/ errors vary. 

Compare the ridge classifier with a logistic regression model. Which one
performs the best?

- Logistic regression gets about the same test score as RidgeClassifier. If I restart the kernel one or the other may be above by 0.01-0.02 but they look to be very similar. 

In [2]:
dataa = "/Users/katherinemottola/Desktop/important/school/GEOL599/1_24_24/LA_AQS_2023.csv" #note on path

df = pd.read_csv(dataa) #Read it in and store it in a variable!

Note - o3 is in ppm, no2 is in ppb, and pm2.5 is in micrograms/cubic meter (LC)

# prepping data

In [3]:
## want to make a df with ozone, no2, and pm2.5

# ozone

dfo3 = df[(df['Parameter Name']=='Ozone') & (df['Duration Description']=='1 HOUR')]

# make a variable of only the mean 
mean_o3 = dfo3["Arithmetic Mean"]

# now make a dataframe where its ONLY the columns of date and ozone 

dfo3 = pd.DataFrame(data = {"O3" : mean_o3, 'date' : dfo3["Date (Local)"]})

In [4]:
# no2

dfno2 = df[(df['Parameter Name']=='Nitrogen dioxide (NO2)') & (df['Duration Description']=='1 HOUR')]

# make a variable of only the mean 
mean_no2 = dfno2["Arithmetic Mean"]

# only date and mean 
dfno2 = pd.DataFrame(data = {"NO2" : mean_no2, 'date' : dfno2["Date (Local)"]})

In [5]:
# pm25

dfpm = df[(df['Parameter Name']=='PM2.5 - Local Conditions') & (df['Duration Description']=='1 HOUR')]

# make a variable of only the mean 
mean_pm = dfpm["Arithmetic Mean"]

# only date and mean 
dfpm = pd.DataFrame(data = {"PM2.5" : mean_pm, 'date' : dfpm["Date (Local)"]})

In [6]:
# now wanna merge the no2 to this, then pm2.5. do it in steps
# no2 first 

df_o3_no2 = dfo3.merge(dfno2, on=['date'])

df_o3_no2.head()

,O3,date,NO2
0,0.031708,2023-01-01,4.816667
1,0.031708,2023-01-01,4.550000
2,0.031708,2023-01-01,4.550000
3,0.031708,2023-01-01,4.816667
4,0.015792,2023-01-02,14.720833


In [7]:
# now pm2.5

df_all = df_o3_no2.merge(dfpm, on=['date'])

df_all.head()

,O3,date,NO2,PM2.5
0,0.031708,2023-01-01,4.816667,5.0875
1,0.031708,2023-01-01,4.550000,5.0875
2,0.031708,2023-01-01,4.550000,5.0875
3,0.031708,2023-01-01,4.816667,5.0875
4,0.015792,2023-01-02,14.720833,9.1875


In [8]:
# get months from 

months = pd.DatetimeIndex(df_all['date']).month.values
months

array([1, 1, 1, ..., 9, 9, 9], dtype=int32)

In [9]:
# now make an array w the same dimensions but where all values are 0 to start with
# in the end itll just basically show whether or not its summer 

# this makes the array thing
qsummer = months - months 

# june july august months, summer = 1
qsummer[months == 6] = 1 
qsummer[months == 7] = 1
qsummer[months == 8] = 1

# march april may, spring is -1 which means we get rid of it
qsummer[months == 3] = -1 
qsummer[months == 4] = -1
qsummer[months == 5] = -1

# also get rid of fall by making it -1
qsummer[months == 9] = -1 
qsummer[months == 10] = -1
qsummer[months == 11] = -1

# this automatically makes winter = 0 bc we're leaving it as-is

In [10]:
df_all['summer'] = qsummer

In [11]:
df_all.head()

,O3,date,NO2,PM2.5,summer
0,0.031708,2023-01-01,4.816667,5.0875,0
1,0.031708,2023-01-01,4.550000,5.0875,0
2,0.031708,2023-01-01,4.550000,5.0875,0
3,0.031708,2023-01-01,4.816667,5.0875,0
4,0.015792,2023-01-02,14.720833,9.1875,0


In [12]:
# get rid of all values where summer is = -1

df_all = df_all[df_all['summer'] > -1]
df_all.describe()

,O3,NO2,PM2.5,summer
count,604.000000,604.000000,604.000000,604.000000
mean,0.025534,14.908291,11.883327,0.609272
std,0.007501,6.819712,4.928007,0.488318
min,0.008333,4.250000,3.250000,0.000000
25%,0.022000,9.700000,8.545833,0.000000
50%,0.026417,13.425000,10.900000,1.000000
75%,0.031542,19.133333,15.266667,1.000000
max,0.042833,33.466667,31.979167,1.000000


In [13]:
# ok. data is mostly set up now. 

In [14]:
# note, y should be the yes or no thing, 
# so in this case y is summer vs winter

In [15]:
y = df_all['summer']
y = y.to_numpy()
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [16]:
X = df_all[['O3','NO2','PM2.5']].copy()
X = X.to_numpy()
X

array([[ 0.031708,  4.816667,  5.0875  ],
       [ 0.031708,  4.55    ,  5.0875  ],
       [ 0.031708,  4.55    ,  5.0875  ],
       ...,
       [ 0.025042, 18.208333, 19.341667],
       [ 0.025042, 20.458333, 19.341667],
       [ 0.025042, 20.458333, 19.341667]])

# onto the models

In [17]:
# Xtrain and xtest are from the air quality variables array (X), ytrain and ytest are from months (y)
# goal: 75 train, 15 for validate, 10 for test

# have to do train test split twice

# first round of ridgeclassifier train test split

In [18]:
# training and remaining instead of train and test, remaining will be what i then pull the next stuff from
X_train, X_temp_remaining, y_train, y_temp_remaining = train_test_split(X, y, test_size=0.25)

# now i can split the remaining data into the testing and validation sets
X_test, X_val, y_test, y_val = train_test_split(X_temp_remaining, y_temp_remaining, test_size=0.4)


# X_temp_train is training data, X_test as testing data, X_val is validation data
# y_temp_train is training data, y_test as testing data, y_val is validation data


In [19]:
month_classifier1 = RidgeClassifier(alpha=0.5)
month_classifier1.fit(X_train, y_train)

month_classifier1.score(X_train, y_train)
train_accuracy = month_classifier1.score(X_train, y_train)
print("training accur score:", train_accuracy)

y_val_pred = month_classifier1.predict(X_val)
val_accuracy = month_classifier1.score(X_val, y_val)
print("validation accur score:", val_accuracy)

y_test_pred = month_classifier1.predict(X_test)
test_accuracy = month_classifier1.score(X_test, y_test)
print("test accur score:", test_accuracy)


training accur score: 0.8498896247240618
validation accur score: 0.819672131147541
test accur score: 0.8333333333333334


- for this try, alpha=0 gets me training score: 0.8587196467991169, validation accur score: 0.8688524590163934 and test accur score: 0.7777777777777778
- 0.1 gets me training score: 0.8631346578366446, validation accur score: 0.8524590163934426 and test accur score: 0.7777777777778
- 0.2 --> training score: 0.8587196467991169, validation accur score: 0.8524590163934426 and test accur score: 0.7777777777778
- 0.3 --> training score: 0.8543046357615894, validation accur score: 0.8524590163934426 and test accur score: 0.7888888888888889
- 0.4 --> same ^
- 0.5 --> same ^
- 0.6 --> same ^
- 0.7 --> same ^
- 0.8 --> same ^
- 0.9 --> same ^
- 1.0 --> same ^

so alpha = 0.1 is best fit for training data

alpha = 0.0 gets the best validation set fit

alpha = 0.3-1.0 get the best accuracy of the model on the test set (but only by like 0.01) 

worst overall is alpha = .3 - 1 for training data and alpha = 0-.2 for test set. Validation set only better w 0.0. .1-1 is same

# redoing splits to see if values are diff

In [20]:
# try 2

# temporary training and remaining instead of train and test, remaining will be what i then pull the next stuff from
X_train2, X_temp_remaining2, y_train2, y_temp_remaining2 = train_test_split(X, y, test_size=0.25)

# Split the remaining data into testing and validation sets
X_test2, X_val2, y_test2, y_val2 = train_test_split(X_temp_remaining2, y_temp_remaining2, test_size=0.4)

In [21]:
month_classifier2 = RidgeClassifier(alpha=1.0)
month_classifier2.fit(X_train2, y_train2)

month_classifier2.score(X_train2, y_train2)
train_accuracy2 = month_classifier2.score(X_train2, y_train2)
print("training accur score:", train_accuracy2)

y_val_pred2 = month_classifier2.predict(X_val2)
val_accuracy2 = month_classifier2.score(X_val2, y_val2)
print("validation accur score:", val_accuracy2)

y_test_pred2 = month_classifier2.predict(X_test2)
test_accuracy2 = month_classifier2.score(X_test2, y_test2)
print("test accur score:", test_accuracy2)

training accur score: 0.8454746136865342
validation accur score: 0.9016393442622951
test accur score: 0.8


- for this try, alpha=0 gets me training score: 0.847682119205298, validation accur score: 0.8524590163934426 and test accur score: 0.8666666666666667
- 0.1 gets me training score: 0.8454746136865342, validation accur score: 0.8360655737704918 and test accur score: 0.8888888888888888
- 0.2 --> same ^
- 0.3 --> same ^
- 0.4 --> training score: 0.8498896247240618, validation accur score: 0.8360655737704918 and test accur score: 0.8888888888888888
- 0.5 --> same ^
- 0.6 --> same ^
- 0.7 --> same ^
- 0.8 --> same ^
- 0.9 --> same ^
- 1.0 --> same ^

so alpha = 0.1 is still best fit for training data

alpha = 0.0 still gets the best validation set fit

alpha = 0.1-1.0 get the best accuracy of the model on the test set not by much 

worst overall is alpha = .1 - 1 

# logistic regression 

In [22]:
logreg = LogisticRegression()  
logreg.fit(X_train, y_train)

train_accuracy = logreg.score(X_train, y_train)
print("train accur score:", train_accuracy)

y_val_pred = logreg.predict(X_val)
val_accuracy = logreg.score(X_val, y_val)
print("validation accur score:", val_accuracy)

y_test_pred = logreg.predict(X_test)
test_accuracy = logreg.score(X_test, y_test)
print("test accur score:", test_accuracy)

train accur score: 0.8741721854304636
validation accur score: 0.8688524590163934
test accur score: 0.8555555555555555
